# Optimizer Agent Notebook

This notebook demonstrates the Optimizer Agent for circuit optimization.

## Purpose
- Test circuit optimization
- Compare optimization strategies
- Evaluate optimization metrics
- Test hardware-specific optimizations

## Usage
Import OptimizerAgent from src.agents.optimizer and optimize quantum circuits.
